In [ ]:
#%pip install numpy pandas scipy plotly scikit-learn lempel_ziv_complexity
#%pip install jupytext
import sys
sys.executable

# permutation entropy
#%pip install ordpy
#%pip install antropy
#%pip install pandas
#%pip install plotly
#%pip install jupyter-dash

In [ ]:
import datetime
import pandas as pd
import numpy as np
import json
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import hilbert 
#from lempel_ziv_complexity import lempel_ziv_complexity
import ordpy
import antropy

from mt_spectrogram import multitaper_spectrogram, nanpow2db


pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 1000)

def load_eeg(json_filename, line_freq=60):
    with open(json_filename) as fp:
        jsn = json.load(fp)
    #print(f'start: {jsn["start_ts"]}, end: {jsn["end_ts"]}, metadata: {jsn["metadata"]}')
    
    dfs = []
    assert jsn["eeg"][0][0]["timestamp"] == jsn["eeg"][1][0]["timestamp"]
    assert jsn["eeg"][0][0]["index"] == jsn["eeg"][1][0]["index"]
    # Should we use the eeg timestamp delta from start_ts? (It's about 40ms)
    print((jsn["start_ts"] - jsn["eeg"][0][0]["timestamp"]) / 1000)
    seq_start = jsn["eeg"][0][0]["index"]
    for chan in jsn["eeg"]:
        for e in chan:
            tmpdf = pd.DataFrame([{"electrode": e["electrode"], "value": s,} 
                                  for i, s in enumerate(e["samples"])])
            relseq_time = (e["index"] - seq_start) * EEG_DT * 12
            tmpdf["reltime"] = [EEG_DT * i + relseq_time for i in range(12)]
            dfs.append(tmpdf)
    eeg_df = pd.concat(dfs).pivot(index="reltime", columns="electrode", values="value")
    electrode_name_map = {i: n for i, n in enumerate(jsn['metadata']['electrodeNames'])}
    eeg_df.rename(columns=electrode_name_map, inplace=True)
    
    # notch-filter eeg
    if line_freq:
        b, a = signal.iirnotch(line_freq, Q=30, fs=EEG_FS)
        for e in eeg_df.columns:
            eeg_df[e] = signal.filtfilt(b, a, eeg_df[e])
    
    dfs = []
    assert jsn["gyro"][0]["sequenceId"] == jsn["accel"][0]["sequenceId"]
    seq_start = jsn["accel"][0]["sequenceId"]
    for a, g in zip(jsn['accel'], jsn['gyro']):
        adf = pd.json_normalize(a["samples"]).rename(columns=dict(x="acc_x", y="acc_y", z="acc_z"))
        gdf = pd.json_normalize(g["samples"]).rename(columns=dict(x="gyr_x", y="gyr_y", z="gyr_z"))
        tmpdf = pd.concat((adf, gdf), axis=1)
        relseq_time = (a["sequenceId"] - seq_start) * MOT_DT * 3
        tmpdf["reltime"] = [MOT_DT * i + relseq_time for i in range(3)]
        dfs.append(tmpdf)
        #tmpdf['samp_offset'] = [samp_offset for i in tmpdf.index]
        #tmpdf['start'] = jsn["start_ts"]
    motion_df = pd.concat(dfs).set_index("reltime")
    
    dfs = []
    # all timestamps are identical for ppg. And they lead the global start_ts by a bit.
    print((jsn["start_ts"] - jsn["ppg"][0]["timestamp"]) / 1000)
    seq_start = jsn["ppg"][0]["index"]
    for p in jsn["ppg"]:
        tmpdf = pd.DataFrame([{"channel": p["ppgChannel"], "ppg": s,} 
                              for i, s in enumerate(p["samples"])])
        relseq_time = (p["index"] - seq_start) * PPG_DT * 6
        tmpdf["reltime"] = [PPG_DT * i + relseq_time for i in range(6)]
        dfs.append(tmpdf)
    ppg_df = pd.concat(dfs).pivot(index="reltime", columns="channel", values="ppg")
    ppg_df.rename(columns={k: f"ppg{k}" for k in ppg_df.columns}, inplace=True)
    
    return jsn['metadata'], eeg_df, motion_df, ppg_df


def calc_bands_power(x, dt, bands):
    f, psd = signal.welch(x, fs=1. / dt)
    power = {band: np.abs(np.mean(psd[np.where((f >= lf) & (f <= hf))])) 
                          for band, (lf, hf) in bands.items()}
    return power


def lzc(x):
    """
    Compute the Lempel-Ziv Complexity on a timeseries x of real numbers.
    This is computed by taking the analytical signal of x (using 
    scipy.signal.hilbert) and creating a series of bits by thresholding
    with meadian of the amplitude.
    """
    h = hilbert(x)
    amp = np.abs(h)
    bitstr = ''.join([str(b) for b in (amp > np.median(amp)).astype(int)])
    complexity = antropy.lziv_complexity(bitstr)
    ph = np.angle(h)
    bitstr = ''.join([str(b) for b in (ph > np.median(ph)).astype(int)])
    ph_complexity = antropy.lziv_complexity(bitstr)
    return complexity, ph_complexity


def logistic(a=4, n=100000, x0=0.4):
    x = np.zeros(n)
    x[0] = x0
    for i in range(n-1):
        x[i+1] = a*x[i]*(1-x[i])
    return(x)

EEG_FS = 256
MOT_FS = 52
PPG_FS = 64
EEG_DT = 1 / EEG_FS
MOT_DT = 1 / MOT_FS
PPG_DT = 1 / PPG_FS
!ls data/Muse*

In [ ]:
# WORK IN PROGRESS
# may be able to simplify data loading for eeg and ppg
def parse_jsn(jsn, dt, nsamp, seq_name="index", chan_name="electrode"):
    dfs = []

    seq_start = jsn[0][seq_name]
    for d in jsn:
        tmpdf = pd.DataFrame([{"chan": d[chan_name], "value": s,} 
                              for i, s in enumerate(d["samples"])])
        relseq_time = (d["index"] - seq_start) * dt * nsamp
        tmpdf["reltime"] = [dt * i + relseq_time for i in range(nsamp)]
        dfs.append(tmpdf)
    df = pd.concat(dfs).pivot(index="reltime", chan_name="electrode", values="value")

In [ ]:
metadata, eeg_df, motion_df, ppg_df = load_eeg(fn)

In [ ]:
px.line(eeg_df)

In [ ]:
px.line(motion_df)

In [ ]:
px.line(ppg_df, y="ppg_0")

In [ ]:


np.random.seed(1234567)
x = np.random.normal(size=3000)
# Permutation entropy
print(antropy.perm_entropy(x, normalize=True))
# Spectral entropy
print(antropy.spectral_entropy(x, sf=100, method='welch', normalize=True))
# Singular value decomposition entropy
print(antropy.svd_entropy(x, normalize=True))
# Approximate entropy
print(antropy.app_entropy(x))
# Sample entropy
print(antropy.sample_entropy(x))
# Hjorth mobility and complexity
print(antropy.hjorth_params(x))
# Number of zero-crossings
print(antropy.num_zerocross(x))
# Lempel-Ziv complexity
print(antropy.lziv_complexity('01111000011001', normalize=True))

In [ ]:
fn = "MuseS-5743_2023-05-05T19_30_08.185Z.json"
fn = f'./data/{fn}'
with open(fn) as fp:
    jsn = json.load(fp)
(jsn.keys(), jsn['accel'][0].keys(), jsn['ppg'][0].keys())

In [ ]:
edf = load_eeg(fn)
EEG_ELECTRODES = edf.electrode.unique()
raw = edf.pivot(index=['reltime'], columns=['electrode'], values=['samp']).reset_index()
raw.columns = [c[1] if c[1] != '' else c[0] for c in raw.columns]
raw.dropna(inplace=True)
raw.head()

In [ ]:
fig = go.Figure()
Sxx, t, f, meta = multitaper_spectrogram(eeg_df.AF7.values, EEG_FS, freq_range=(0, 40), ncores=-1)

fig.add_trace(go.Heatmap(x=t, y=f, z=Sxx.clip(-5, 5), colorscale='Solar'))
fig.update_layout(title='Average Multitaper Spectrogram', 
                  font=dict(size=18),
                  yaxis=dict(title='Frequency (Hz)'), 
                  xaxis=dict(title='Time from start (seconds)'),
                  width=900, height=500)
fig

In [ ]:
px.line(motion_df, y=["gyr_x", "gyr_y", "gyr_z"])

In [ ]:
ca, cp = lzc(raw.AF7)
print(ca, cp)

In [ ]:
time_series = [logistic(a) for a in [3.05, 3.55, 4]]
time_series += [np.random.normal(size=100000)]

HC = [ordpy.complexity_entropy(series, dx=4) for series in time_series]
HC

In [ ]:
#ordpy.permutation_entropy?
#ordpy.complexity_entropy?

In [ ]:
n = 1000
x = np.sin(np.linspace(0, 100 * np.pi, n)) + np.random.randn(n) * 0.0
c = antropy.lziv_complexity(x)
ce = ordpy.complexity_entropy(x)
print(c, ce)

In [ ]:
# sequenceId is the "timestamp" and 
tmpdf = pd.json_normalize(jsn['accel'])
tmpdf = tmpdf.explode(column=['samples']) #.reset_index(drop=True)
tmpdf.head()

In [ ]:
fns = {'cris': './data/MuseS-4DD2_1680820272258.json', 
       'bob': './data/MuseS-5743_1680820272258.json',
       'patrick': './data/MuseS-5C4F_1680820272258.json'}

dfs = []
for uid, fn in fns.items():
    tmpdf = load_eeg(fn)
    #tmpdf["user_id"] = uid
    dfs.append(tmpdf)

dfs[0].head()

In [ ]:
idx = 1
EEG_ELECTRODES = dfs[idx].electrode.unique()
raw = dfs[idx].pivot(index=['reltime'], columns=['electrode'], values=['samp']).reset_index()
raw.columns = [c[1] if c[1] != '' else c[0] for c in raw.columns]
raw.dropna(inplace=True)
raw.head()

In [ ]:
win = int(round(8 * EEG_FS))
stepwin = int(round(1 * EEG_FS))
y = raw[EEG_ELECTRODES].rolling(window=win, center=True, step=stepwin).apply(lzc)
y["reltime"] = raw.reltime.groupby(raw.index // stepwin).mean()
px.line(y, x="reltime", y=[])

In [ ]:
bands = {#'Delta': (0, 4),
         'Theta': (4, 8),
         'Alpha': (8, 12),
         'Beta': (12, 30),
         'Gamma': (30, 55),
         'High-gamma': (65, 100)}

eeg_pow = calc_bands_power(raw.AF7, EEG_DT, bands)
fig = go.Figure(go.Bar(x=[v for v in eeg_pow.values()], y=[k for k in eeg_pow], orientation='h'))
fig.show()

In [ ]:
fig = go.Figure()
Sxx, t, f, meta = multitaper_spectrogram(raw.AF7.values, EEG_FS, freq_range=(0, 120), ncores=-1)

fig.add_trace(go.Heatmap(x=t, y=f, z=Sxx.clip(-5, 5), colorscale='Solar'))
fig.update_layout(title='Average Multitaper Spectrogram', 
                  font=dict(size=18),
                  yaxis=dict(title='Frequency (Hz)'), 
                  xaxis=dict(title='Time from start (seconds)'))
fig

In [ ]:
bands = {#'Delta': (0, 4),
         'Theta': (4, 8),
         'Alpha': (8, 12),
         'Beta': (12, 30),
         'Gamma': (30, 55),
         'High-gamma': (65, 100)}

eeg_pow = calc_bands_power(raw.AF7, EEG_DT, bands)
fig = go.Figure(go.Bar(x=[v for v in eeg_pow.values()], y=[k for k in eeg_pow], orientation='h'))
fig.show()

In [ ]:
fig = go.Figure()
Sxx, t, f, meta = multitaper_spectrogram(raw.AF7.values, EEG_FS, freq_range=(0, 120), ncores=-1)

fig.add_trace(go.Heatmap(x=t, y=f, z=Sxx.clip(-5, 5), colorscale='Solar'))
fig.update_layout(title='Average Multitaper Spectrogram', 
                  font=dict(size=18),
                  yaxis=dict(title='Frequency (Hz)'), 
                  xaxis=dict(title='Time from start (seconds)'))
fig

In [ ]:
NPERSEG = 64
#IDX = (20.0, 120.0, 145.0, 300.0)
IDX = (1.0, 20.0, 25.0, 50.0)

fig = go.Figure()
idx = (raw.reltime > IDX[0]) & (raw.reltime < IDX[1])
f, Cxy = signal.coherence(raw.AF7[idx] + raw.TP9[idx], raw.AF8[idx] + raw.TP10[idx], 256, nperseg=NPERSEG)
fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Task'))
idx = (raw.reltime > IDX[2]) & (raw.reltime < IDX[3])
f, Cxy = signal.coherence(raw.AF7[idx] + raw.TP9[idx], raw.AF8[idx] + raw.TP10[idx], 256, nperseg=NPERSEG)
fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Rest'))
    
fig.update_layout(yaxis= {'type': 'log', 'title': 'Coherence'},
                  xaxis_title='Frequency',
                  legend={'font': {'size': 14}, 
                          #'title': {'font': {'size': 16}, 'text': 'Measure'},
                          'yanchor': 'bottom', 'y': 0.05, 'xanchor': 'center', 'x': 0.5},
                  title='Fronto-temporal Coherence',
                  font={'size': 18})

fig.show()

In [ ]:
NPERSEG = 64
#IDX = (20.0, 120.0, 145.0, 300.0)
IDX = (1.0, 16.0, 22.0, 55.0)

fig = go.Figure()
for k in [('E0', 'E1'), ('E3', 'E2')]:
    idx = (raw.reltime > IDX[0]) & (raw.reltime < IDX[1])
    f, Cxy = signal.coherence(raw[k[0]][idx], raw[k[1]][idx], 256, nperseg=NPERSEG)
    fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Task {k[0]} v. {k[1]}'))
    idx = (raw.reltime > IDX[2]) & (raw.reltime < IDX[3])
    f, Cxy = signal.coherence(raw[k[0]][idx], raw[k[1]][idx], 256, nperseg=NPERSEG)
    fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Rest {k[0]} v. {k[1]}'))
    
fig.update_layout(yaxis= {'type': 'log', 'title': 'Coherence'},
                  xaxis_title='Frequency',
                  legend={'font': {'size': 14}, 
                          #'title': {'font': {'size': 16}, 'text': 'Measure'},
                          'yanchor': 'bottom', 'y': 0.05, 'xanchor': 'center', 'x': 0.5},
                  title='Coherence',
                  font={'size': 18})

fig.show()

In [ ]:
from numpy_ext import rolling_apply

def coherence(x, y):
    f, Cxy = signal.coherence(x, y, 256, nperseg=NPERSEG)
    return f, Cxy

df = raw.copy().set_index('samp')

#df[['f', 'Cxy']] = rolling_apply(coherence, , df.AF7.values, df.TP9.values)
#locdf[['dist', 'bearing']] = pd.DataFrame(np.row_stack(np.vectorize(dist_az, otypes=['O'])(
#    locdf['latitude'], locdf['longitude'], locdf['homelat'], locdf['homelon'])), index=locdf.index)
#print(df)

In [ ]:
fig = make_subplots(rows=3, cols=1, subplot_titles=('Sensors', 'Raw EEG', 'Muse Bands'))
#fig = go.Figure(go.Bar(y=statdf.index, x=statdf['User-days'], orientation='h'))

for v in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x=acc.samp, y=acc[v], name=f'Accel {v.upper()}'), row=1, col=1)
    fig.add_trace(go.Scatter(x=gyr.samp, y=gyr[v], name=f'Gyro {v.upper()}', yaxis='y2'), row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=1)
fig.update_yaxes(title_text="Accelerometer (m/s/s)", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Gyro (rad/s)", row=1, col=1, secondary_y=True, anchor='x',
                 overlaying='y', side='right')

for v in ['TP9', 'AF7', 'AF8', 'TP10']: #, 'Aux']:
    fig.add_trace(go.Scatter(x=raw.samp, y=raw[v], name=f'{v.upper()}', opacity=0.5), row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=1)

#for v in ['delta', 'theta', 'alpha', 'beta', 'gamma']:
#    tmp = band.loc[bands.band == v, :].copy().reset_index()
#    tmp['samp'] = tmp.index / SEN_FS
#    fig.add_trace(go.Scatter(x=tmp.samp, y=tmp.AF7 + tmp.AF8 + tmp.TP9 + tmp.TP10, name=f'{v}'), row=3, col=1)
#fig.update_xaxes(title_text="Time", row=3, col=1)

fig.update_layout(height=1000, 
                  title='Muse EEG', 
                  #showlegend=False,
                  font={'size': 18})

fig.show() 

In [ ]:
from sklearn.feature_selection import mutual_info_regression as MIR

N = 10000
X = np.random.randn(N, 1)
y = X[:, 0] +  np.random.randn(N) * 0.01
mi_score = MIR(X, y)
print(mi_score)